This Notebook will be used for the Coursera IBM Data science Capstone project


1. Introduction/Business Problem- Introduction where you discuss the business problem and who would be interested in this project.
2. Data- where you describe the data that will be used to solve the problem and the source of the data.
3. Methodology- section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
4. Results- section where you discuss the results.
5. Discussion- section where you discuss any observations you noted and any recommendations you can make based on the results.
6. Conclusion- section where you conclude the report.


<b>Introduction/Business Problem-</b> A description of the problem and a discussion of the background.
An entrepreneur wants to open an restaurant in the Downtown Toronto area. There are several Neighborhoods in the Downtomwn Toronto area. The ideal location will be the one that already has different types of restaurants. The objective is to come up with the top 3 types of Restaurants in that neighborhood. This analysis will also help any entreprenuer to determine the ideal location depending on the type of the restaurant that is proposed to be opened.

<b>Data-</b> A description of the data and how it will be used to solve the problem. (15 marks)
The data collection should enable to decide the top3 Restaurants in each neighborhood. So a getting a list of establishments for each neighborhood in the downtown toronto area will enable to find the top 3 Restaurants. 1> The Wikipedia page has a list of Canada postal codes and Boroughs and Neighborhoods. This data will have to be scrapped fomr the web page. 2> Retrieve / Get the lsit of Geo coordinates (Latitude and longitude) for each postal code. 3> Use the Foursuare API to get the list of Venues for each Latitude and Longitude. 4> Construct a normalized Dataframe by converting the Categorical Values of the Venue categories to columns having numeric values. 5> Use Kmeans clustering to find the clusters with are similar. 6> These clusters will let the entrprenuer know the primary activity in that cluster and help him to take a decision on the location. For ex. if the entreprenuer wants to service office employees, the restaurant can be opened in a boourgh which has business activity.

<b>Methodology-</b>

In [22]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library
#!pip install geocoder
print('Libraries imported.')

Libraries imported.


In [14]:
# Refer to the follwing link where the data scarpping was done and saved in a csv:https://dataplatform.cloud.ibm.com/analytics/notebooks/v2/859bdf57-8c48-41c1-9317-390fba92c940/view?access_token=d2f78b0a79f98b74fc3899372dbac2f192d83c4eb320e04a0b0ba16ab7841c10
import pandas as pd
from project_lib import Project
project = Project(project_id='6c00565f-d54b-4419-8ea0-5422c90cb00a', project_access_token='p-4c2fc97e8b68c62358d8689ba87a5c0548556576')
my_file = project.get_file("candntorontovenues.csv")
my_file.seek(0)

dn_toronto_data_venues=pd.read_csv(my_file)
dn_toronto_data_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [15]:
dn_toronto_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [16]:
print('There are {} uniques categories.'.format(len(dn_toronto_data_venues['Venue Category'].unique())))

There are 209 uniques categories.


In [17]:
# one hot encoding- This is to convet the Categories in to columns havinf numeric values.
dn_toronto_onehot = pd.get_dummies(dn_toronto_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dn_toronto_onehot['Neighborhood'] = dn_toronto_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dn_toronto_onehot.columns[-1]] + list(dn_toronto_onehot.columns[:-1])
dn_toronto_onehot = dn_toronto_onehot[fixed_columns]

dn_toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
dn_toronto_onehot.shape

(1233, 209)

In [19]:
dn_toronto_grouped = dn_toronto_onehot.groupby('Neighborhood').mean().reset_index()
dn_toronto_grouped.shape

(19, 209)

In [68]:
num_top_venues = 5

for hood in dn_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dn_toronto_grouped[dn_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2          Restaurant  0.03
3  Seafood Restaurant  0.03
4              Bakery  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3      Nightclub  0.06
4     Restaurant  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3           Restaurant  0.04
4        

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dn_toronto_grouped['Neighborhood']

for ind in np.arange(dn_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dn_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Gourmet Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Plane,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Bar
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Hotel,Café,Yoga Studio,Mediterranean Restaurant,Men's Store
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Italian Restaurant
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Seafood Restaurant,Asian Restaurant,Steakhouse,Deli / Bodega
7,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bookstore
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Baseball Stadium
9,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Farmers Market,Dessert Shop,Comfort Food Restaurant,Caribbean Restaurant,Gaming Cafe


# Clustering Kmeans

In [53]:
# set number of clusters
kclusters = 5 # This will create 5 clusters

dn_toronto_grouped_clustering = dn_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dn_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4], dtype=int32)

In [60]:
# Get the saved file
canpostalgeocd_file = project.get_file("canpostalgeocd-dn.csv")
canpostalgeocd_file.seek(0)

toronto_data=pd.read_csv(canpostalgeocd_file)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [69]:
# add clustering labels
# labels are already inserted in the above code.
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dn_toronto_merged = toronto_data
# merge dn_toronto_merged with toronto_data to add latitude/longitude for each neighborhood
dn_toronto_merged = dn_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
dn_toronto_merged.head() # check the last columns!
#dn_toronto_merged.shape

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Café,Bakery,Pub,Breakfast Spot,Theater,Restaurant,Yoga Studio,Farmers Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Italian Restaurant,Japanese Restaurant,Music Venue,Sandwich Place,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Department Store,Creperie,Cosmetics Shop,Gym,Clothing Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Gourmet Shop


In [62]:
# Get the Latitude and longitude of Downtown Toronto, Country code CA
address = 'Downtown Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
dn_toronto_latitude = location.latitude
dn_toronto_longitude = location.longitude
print('The geographical coordinate of Downtown Toronto are {}, {}.'.format(dn_toronto_latitude, dn_toronto_longitude))

The geographical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [63]:
# create map
map_clusters = folium.Map(location=[dn_toronto_latitude, dn_toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dn_toronto_merged['Latitude'], dn_toronto_merged['Longitude'], dn_toronto_merged['Neighborhood'],dn_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Print the Clusters

In [72]:
# get the 2nd column (index=1) and columns from 5 to the total number of columns (dn_toronto_merged.shape[1])
# do this for all 5 clusters.
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 0, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Plane,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Bar


In [73]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 1, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Park,Café,Bakery,Pub,Breakfast Spot,Theater,Restaurant,Yoga Studio,Farmers Market
1,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Italian Restaurant,Japanese Restaurant,Music Venue,Sandwich Place,Distribution Center
2,"Garden District, Ryerson",1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bookstore
3,St. James Town,1,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Department Store,Creperie,Cosmetics Shop,Gym,Clothing Store
4,Berczy Park,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Gourmet Shop
5,Central Bay Street,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant
7,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Thai Restaurant,Gym,Pizza Place,Clothing Store,Bookstore
8,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Baseball Stadium
9,"Toronto Dominion Centre, Design Exchange",1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Asian Restaurant
10,"Commerce Court, Victoria Hotel",1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Italian Restaurant


In [74]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 2, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,2,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop


In [75]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 3, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,3,Park,Trail,Playground,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [76]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 4, dn_toronto_merged.columns[[2] + list(range(5, dn_toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"University of Toronto, Harbord",4,Café,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Yoga Studio,Beer Bar,College Gym,Sandwich Place
12,"Kensington Market, Chinatown, Grange Park",4,Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Farmers Market,Dessert Shop,Comfort Food Restaurant,Caribbean Restaurant,Gaming Cafe


<b>Results-</b>
The Results display the top 10 most common venues for each neighborhood. 
As can be seen from the results the Cluster1 has restaurants as all the top10 venues.
We used K-means clustering to group the neighborhood in to clusters.
The map shows that the Cluster1 (dark brown in color) has neighborhoods spread out through out the Downton Toronto area.


<b>Discussion-</b> As can be seen from the results the Cluster2 has restaurants as all the top10 venues. There are lot of neighborhood choices in CLuster2.  
The restaurant can be opened in any of these neighborhoods in Cluster2.
The other factors that would affect the decision are:
1. The type/ Cuisine of the restaurant to be opened.
2. The clientile the restaurant wants to serve, for ex if the restaurant wants to target college students, Cluster5 which has university students will be a suitable location.
3. The budget of the business, this means factors such as rent and other costs have to be considered.
4. Finally, other factors affecting the decision would be familiarity of the entreprenuer with the neighborhood, closeness of the neighborhood for the entreprenuer.

<b>Conclusion-</b> The report defines the Business problem and then identifies the data necessary that will help in making recommendations.
The data is formatted in a manner suitable for the analysis.
The analysis is done by narrowing the data doen to top10 venues for each neighborhood and using K-means clustering which enables the identification of similar neighborhoods.
These results and also other factors mentioned in the Discussion section will be considered in making a recommedation.
The Recommedation will definitely help the entreprenuer in taking a good decision about the type of resataurant, the location whihc also considers the finaces avaliable and set the entreprenuer on the path to Success!.